In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003767251968383789
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 12.591494977574651
200000 24.26254872734333
300000 23.909092663274095
400000 23.974282767890678
500000 20.724119759715354
600000 24.651993725657114
700000 23.369925025646452
800000 23.38933916824667
900000 20.058552089227206
1000000 20.58491143568957
1100000 8.782378776419272
1200000 21.093431683215332
1300000 18.23689852324277
1400000 18.183252751747244
1500000 19.79327454297693
1600000 19.45397510549115
1700000 19.976488286955657
1800000 18.540354702179698
1900000 20.280838848961004
2000000 19.09278772320248
2100000 5.009211869053023
2200000 23.309964410436027
2300000 23.24111373963154
2400000 24.134055345251454
2500000 23.033261327274108
2600000 24.770476530191182
2700000 22.974942895756854
2800000 22.420140778263715
2900000 23.290072596159543
3000000 2

29400000 16.557612049152723
29500000 17.86858171770087
29600000 16.01325756681245
29700000 11.340154060577806
29800000 12.366731228390805
29900000 16.039622748219745
30000000 16.426427423985807
30100000 17.544237420290738
30200000 17.66769899102395
30300000 18.531174931906286
30400000 17.978447034313657
30500000 12.976931625324848
30600000 13.983577159861117
30700000 17.882993364464937
30800000 19.09049376111266
30900000 17.96551473367339
31000000 10.66623956371625
31100000 15.858275213540125
31200000 19.339390167175736
31300000 18.00575010457366
31400000 18.64595352072779
31500000 17.15899565423428
31600000 16.09602199709858
31700000 17.832345039090324
31800000 5.384639655963045
31900000 16.935869231501947
32000000 16.940368519195168
32100000 16.67623296024635
32200000 18.22497794257558
32300000 17.804359471327157
32400000 18.773053083658517
32500000 15.59299611929402
32600000 10.157382484435097
32700000 18.56588942576537
32800000 17.630147082633275
32900000 19.169508685219746
3300000

59100000 16.034354235602642
59200000 15.426210619134482
59300000 16.507607865411604
59400000 15.644071577194081
59500000 17.307855248607822
59600000 15.696378889545624
59700000 16.78885754638201
59800000 15.332580506139786
59900000 8.566021181592184
60000000 13.969670487589713
60100000 13.942875353709779
60200000 15.50146215325773
60300000 13.986119807629837
60400000 12.926163564710054
60500000 7.858695356518792
60600000 13.923029952317654
60700000 14.654002435149895
60800000 13.838072049314853
60900000 13.67434958116142
61000000 9.477637855020024
61100000 13.86762464168234
61200000 16.016885523875843
61300000 15.447922274635681
61400000 16.1009043622566
61500000 17.142055081003
61600000 15.857180348118582
61700000 15.911383875030868
61800000 15.306364289695304
61900000 16.27868515435036
62000000 6.264843690889506
62100000 14.130590353412181
62200000 14.080905114297614
62300000 13.359802196694863
62400000 14.557945657913391
62500000 6.657263042373696
62600000 12.679170896754098
6270000

88700000 12.004266953985752
88800000 7.241953821103161
88900000 11.209643976088422
89000000 13.085130642833073
89100000 8.740084304917053
89200000 8.076318224225055
89300000 12.135147489950285
89400000 11.302480339393721
89500000 7.036064565394697
89600000 12.045781059235798
89700000 11.757233395984205
89800000 12.348914131205426
89900000 13.03109142928644
90000000 13.283701497290691
90100000 13.27246137998214
90200000 13.113761195060144
90300000 13.34257401295047
90400000 12.75283970174442
90500000 12.725135210079038
90600000 12.055012350774607
90700000 11.748849902230543
90800000 11.991124686821045
90900000 4.992481745758555
91000000 11.838072746079703
91100000 11.78581696057603
91200000 3.9736384524754516
91300000 11.844242585152365
91400000 11.808072203393937
91500000 5.678798540552505
91600000 12.244292707974342
91700000 11.757344312645936
91800000 11.640415652018932
91900000 12.56177438563458
92000000 12.592160437071819
92100000 13.01059611706291
92200000 12.982287995116808
92300

117900000 4.037289768372299
118000000 8.854186770953484
118100000 9.684260943599218
118200000 9.611327660188397
118300000 9.533926925275518
118400000 9.837316587858558
118500000 9.741432709600451
118600000 9.936199491078106
118700000 9.92909686223765
118800000 10.607950981672317
118900000 10.657734424635859
119000000 9.536483075209063
119100000 9.56323397929583
119200000 9.143072073149726
119300000 9.153040025855791
119400000 9.313367059822072
119500000 9.318695510522899
119600000 8.948544967101933
119700000 9.271899062458296
119800000 3.120285977361386
119900000 1.7409242395631692
120000000 6.728222746804682
120100000 8.884743212078567
120200000 8.983204506306983
120300000 9.273550380836516
120400000 9.20989106119704
120500000 9.26080513314426
120600000 9.124294271578574
120700000 9.580069040388857
120800000 9.626617539803439
120900000 11.167443263930252
121000000 10.711498955311054
121100000 10.370089160925072
121200000 10.11562472387251
121300000 9.80649889001595
121400000 10.037631

146800000 7.708282786999325
146900000 7.914761115294854
147000000 7.966547910331836
147100000 8.112721105190266
147200000 7.957942321893024
147300000 7.863755769708282
147400000 8.602206214645063
147500000 8.54232936844087
147600000 8.798646154245606
147700000 8.129341111981145
147800000 7.733924481954558
147900000 7.798996768893687
148000000 7.742119290414613
148100000 7.734286254256349
148200000 7.604479247052422
148300000 7.309467765881494
148400000 3.753833970758631
148500000 6.192382840902795
148600000 5.067140814120639
148700000 3.328007880125282
148800000 7.50495521888496
148900000 7.575969847745726
149000000 7.752996550416077
149100000 7.727234321609469
149200000 7.723623615159004
149300000 7.68545655926809
149400000 8.028184107703584
149500000 8.559754623995037
149600000 8.419094336536773
149700000 8.419797509168681
149800000 7.723717109361992
149900000 7.777594902039938
150000000 7.737868758479776
150100000 7.758854824737683
150200000 7.708688467584045
150300000 7.45597523203

176100000 6.453142830707654
176200000 6.385613577830395
176300000 6.225300198841393
176400000 6.309209698361927
176500000 6.0916386296008955
176600000 5.971092032305236
176700000 5.648372342227137
176800000 4.660768176991074
176900000 3.7642185746583756
177000000 5.6997743646686505
177100000 5.714082041801594
177200000 3.096752926332432
177300000 4.7178542082224615
177400000 5.622789285135687
177500000 5.688969764566265
177600000 2.4395301419742683
177700000 5.814640806417485
177800000 5.675461793486262
177900000 6.24683577490567
178000000 6.042204562471082
178100000 6.295423290943536
178200000 6.316097884230186
178300000 6.231489068506631
178400000 6.090367787150302
178500000 6.059430288071809
178600000 6.279173011041176
178700000 5.689455936819267
178800000 5.606991316772497
178900000 2.689155014186386
179000000 5.6771447193232225
179100000 5.721014568802815
179200000 5.572561305868256
179300000 2.627655779767464
179400000 5.672643895226248
179500000 5.865410413522533
179600000 4.900

205300000 2.0408366584299937
205400000 4.012714319600843
205500000 3.9824588176368603
205600000 3.842075107207215
205700000 3.777023921823603
205800000 3.745349126258958
205900000 1.4163540104331553
206000000 4.005649774255896
206100000 3.782647627046856
206200000 3.7528284198848025
206300000 3.8807698093586964
206400000 4.062615044197948
206500000 1.7212909828516296
206600000 4.328847573527223
206700000 4.268426447083485
206800000 4.2958201602734905
206900000 4.19553500561651
207000000 4.311933357072958
207100000 4.253977655919078
207200000 4.40309024543439
207300000 2.855995369969375
207400000 3.093420051694229
207500000 3.9973888406623597
207600000 3.900401611570508
207700000 3.6847632044765186
207800000 3.7233516613678517
207900000 2.941682619950572
208000000 2.3710436737715606
208100000 3.7473373430337924
208200000 3.714515315762737
208300000 3.8926324081999053
208400000 3.937176000636858
208500000 3.703576981988952
208600000 2.186317731762088
208700000 4.218683236653711
208800000

234100000 2.098527871229688
234200000 2.1253358854553124
234300000 2.022543009424668
234400000 1.984050633509216
234500000 1.9924283174737456
234600000 0.7057681230621364
234700000 1.9274290918176777
234800000 1.9453034235415916
234900000 2.0338446577277103
235000000 2.0449679670383083
235100000 2.0667882201019685
235200000 2.1115554083790777
235300000 2.0959430839069113
235400000 0.8903380880268453
235500000 2.1255953297131964
235600000 2.1558499062394643
235700000 2.0547474130601646
235800000 0.9021443118895822
235900000 2.040044202688995
236000000 1.9950322723575056
236100000 1.9753316511509622
236200000 1.9047966833865595
236300000 1.9847550781808674
236400000 1.908986352216336
236500000 1.8409277148061263
236600000 1.0466487091421286
236700000 1.4975727890833272
236800000 1.8217120218239402
236900000 1.8749687689627468
237000000 1.984656582082457
237100000 1.9144715797589116
237200000 1.9394041177818357
237300000 1.93780236006779
237400000 1.7956015712623117
237500000 1.0092981391

262100000 0.0007232249586919945
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 6.103507976531983e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
